# Exercise 3.2 Movement of a robot using velocity commands

## Theoretical background

In the remaining assignments related to robot motion we will describe two probabilistic motion models for planar movement. The *velocity motion model* and the *odometry motion model*, the former being the main topic of this assignment.

The *velocity motion model* it is used mainly for motion planning, where the details of the robot's movement are of importance and odometry information is not available (it is computed after the movement).

This motion model is characterized by the use of two velocities to control the robot's movement: *linear velocity* $v$ and *angular velocity* $w$. Therefore, during the following exercises, the movement commands will be of the form: $$u = \begin{bmatrix} v \\ w \end{bmatrix}$$

In [ ]:
%matplotlib notebook

# IMPORTS

import numpy as np
from numpy import random
import matplotlib.pyplot as plt

from utils.DrawRobot import DrawRobot
from utils.PlotEllipse import PlotEllipse
from utils.tcomp import tcomp

## 3.2.1

**Context**

This motion model is characterized by the following equations:
<table>
    <tr>
        <td>

- If $w \neq 0$:

    $$
        x_t = x_{t-1} + 
        \begin{bmatrix}
            -R \sin \theta_{t-1} + + R \sin(\theta_{t-1} + \Delta \theta) \\ 
            R \cos \theta_{t-1} - R \cos(\theta_{t-1} + \Delta \theta)\\
            \Delta \theta
        \end{bmatrix}
    $$

- If $w = 0$:

    $$
        x_t = x_{t-1} + 
        \begin{bmatrix}
            \cos \theta_{t-1} \\ \sin \theta_{t-1} \\ 0
        \end{bmatrix}
    $$
    <td/>       
    <td>
        $$ 
        \begin{aligned}
        v &= w \cdot R \\
        \Delta \theta &= w \cdot  \partial t
        \end{aligned}
        $$
    <td/>
  <tr/>
<table/>
    
**Assignment**

Modify the following `main()`, introducing an if-else statement that takes into account when the robot moves in an straight line ($w = 0$). At this point we don't take into account uncertainty in the system: neither from the initial pose (matrix $P_{3\times3}$) nor the movement $(v, w)$ (matrix $Q_{2\times2}$).

**Example**

<figure style="text-align:center">
  <img src="images/fig3-2-1.png" alt="">
  <figcaption>Fig. 1: Route of our robot.</figcaption>
</figure>

**Auxiliary code**

To simplify the demo we'll provide the following auxuliary code.

In [ ]:
def next_pose(x, u, dt, cov=None):
    ''' This function takes pose x and transform it according to the motion u=[v,w]'
        applying the differential drive model.

        Args:
            x: current pose
            u: differential command as a vector [v, w]'
            dt: Time interval in which the movement occurs
            cov: covariance of our movement. If not None, then add gaussian noise
    '''
    if cov is not None:
        u += cov @ random.randn(2, 1)

    if u[1] == 0: #linear motion w=0. Only motion in x
        #dx = u(1)*dT; dy = 0; d_thetha = 0;
        y = np.vstack([x[0] + u[0]*dt*np.cos(x[2]),
                       x[1] + u[0]*dt*np.sin(x[2]),
                       x[2]])
    else: #Non-linear motion w=!0
        R = u[0]/u[1] #v/w=r is the curvature radius
        y = np.vstack([x[0] - R*np.sin(x[2]) + R*np.sin(x[2]+u[1]*dt),
                       x[1] + R*np.cos(x[2]) - R*np.cos(x[2]+u[1]*dt),
                       x[2] + u[1]*dt])

    return y

class VelocityRobot(object):
    """ Mobile robot implementation that uses velocity commands.
    
        Attr:
            pose: expected pose of the robot in the real world (without taking account noise)
            dt: Duration of each step in seconds
    """
    
    def __init__(self, mean, dt):
        self.pose = mean
        self.dt = dt
    def step(self, u):
        self.pose = next_pose(self.pose, u, self.dt)
    def draw(self, fig, ax):
        DrawRobot(fig, ax, self.pose)

Test the movement of your robot using the demo below.

In [ ]:
def main(robot, nSteps):
          
    v = 1 # Linear Velocity 
    l = 0.5 #Half the width of the robot
        
    # MATPLOTLIB
    fig, ax = plt.subplots()
    plt.ion()
    fig.canvas.draw()
    plt.xlim((-2, 20))
    plt.ylim((-2, 30))
    
    plt.grid()
        
    # MAIN LOOP
    for k in range(1, nSteps + 1):
        #control is a wiggle with constant linear velocity
        u = np.vstack((v, np.pi / 10 * np.sin(4 * np.pi * k/nSteps)))
        
        robot.step(u)   
        
        #draw occasionally
        if (k-1)%20 == 0:
            robot.draw(fig, ax)
            fig.canvas.draw()
            plt.pause(0.1)


# RUN 
dT = 0.1 # time steps size
pose = np.vstack([0., 0., 0.])

robot = VelocityRobot(pose, dT)
main(robot, nSteps=400)

## 3.2.2

**Context**

Now we will include uncertainty to the previous exercise, changing the behaviour of the robot class you have implemented.

In contrast to the noisy robot in practice 3-1, we will use the equations of the velocity motion model and their respective Jacobians to keep track of how confident we are of the robot's pose (i.e. the robot's pose now is also a gaussian distribution).

Therefore, we have to deal with 2 Gaussian dists.: the **pose** $(x, P)$ and the **movement command** $(u, Q)$, the latter being applied during an interval of time $\partial t$.

The covariance of this movement $(Q)$ is defined as seen below. It is constant throughout the execution of our code

$$
    Q = \begin{bmatrix}
            \sigma_v^2 & 0 \\
            0 &  \sigma_w^2
        \end{bmatrix}
$$

Whereas, the pose's covariance $(P_t)$ has do be updated at every step of the execution. To achieve this you'll have to use:

$$P_t = JacF_x \cdot P_{t-1} \cdot JacF_x^T + JacF_u \cdot Q \cdot JacF_u^T$$
 
Where $JacF_x$ and $JacF_u$ are the jacobians of our motion model, given the pose $x$ and the action $u$.
Derivate the ecuation of this jacobians and check them against the solution in the apendix of the slides.
      
**Assignment**
      
1. Complete the following code: calculating the covariance matrix $P_k$ and then draw them as ellipses.
 

 
**Example**
 
<figure style="text-align:center">
  <img src="images/fig3-2-2.png" alt="">
  <figcaption>Fig. 2: Movement of a robot using velocity commands. <br/> Representing the expected pose (in red), the true pose (as dots) <br/> and the confidence ellipse.</figcaption>
</figure>

In [ ]:
def next_covariance(x, P, Q, u, dt):
    ''' Compute the covariance of a robot following the velocity motion model

        Args:
            x: current pose (before movement)
            u: differential command as a vector [v, w]''
            dt: Time interval in which the movement occurs
            P: current covariance of the pose
            Q: covariance of our movement.
    '''
    # Aliases
    v = u[0, 0]
    w = u[1, 0]

    sx, cx = np.sin(x[2, 0]), np.cos(x[2, 0]) #sin and cos for the previous robot heading
    si, ci = np.sin(u[1, 0]*dt), np.cos(u[1, 0]*dt) #sin and cos for the heading increment
    R = u[0, 0]/u[1, 0] #v/w Curvature radius

    if u[1, 0] == 0:  #linear motion w=0 --> R = infinite
        #TODO JACOBIAN HERE
        raise NotImplementedError
        JacF_x = None
        JacF_u = None
    else: #Non-linear motion w=!0
        # TODO JACOBIAN HERE
        raise NotImplementedError
        JacF_x = None
        JacF_u = None
    #prediction steps
    return (JacF_x@P@JacF_x.T)+(JacF_u@Q@JacF_u.T)

2. Draw a mark in the poses corresponding to the ground-truth, which need to be randomly generated from the $Q$ matrix. Then test it using your previous `main()`.

In [ ]:
class NoisyVelocityRobot(VelocityRobot):
    """ Mobile robot implementation that uses velocity commands.
       
        Attr:
            [...]: Inherited from VelocityRobot
            true_pose: expected pose of the robot in the real world (noisy)
            cov_pose: Covariance of the pose at each step
            cov_move: Covariance of each movement. It is a constant

    """
    
    def __init__(self, mean, cov_pose, cov_move, dt):
        super().__init__(mean, dt)
        self.true_pose = mean
        self.cov_pose = cov_pose
        self.cov_move = cov_move
        
    def step(self, u):
        self.cov_pose = next_covariance(self.pose, self.cov_pose, self.cov_move, u, self.dt)
        
        super().step(u)
        self.true_pose = next_pose(self.true_pose, u, self.dt, cov=self.cov_move)
        
    def draw(self, fig, ax):
        super().draw(fig, ax)
        el = PlotEllipse(fig, ax, self.pose, self.cov_pose)
        ax.plot(self.true_pose[0, 0], self.true_pose[1, 0], 'o', color=el[0].get_color())

# RUN
dT = 0.1 # time steps size

SigmaV = 0.1 #Standard deviation of the linear velocity. 
SigmaW = 0.1 #Standard deviation of the angular velocity
nSteps = 400 #Number of motions

P = np.diag([0.2, 0.4, 0.]) #pose covariance matrix 3x3
Q = np.diag([SigmaV**2, SigmaW**2]) #motion covariance matrix 2x2

robot = NoisyVelocityRobot(np.vstack([0., 0., 0.]), P, Q, dT)
main(robot, nSteps=nSteps)